<strong>В пространстве $C[0; 1]$ дано интегральное уравнение Фредгольма с вырожденным ядром, содержащее числовой параметр $\lambda > 0$.

A. Определить, при каких значениях параметра $\lambda$ к этому уравнению применим принцип сжимающих операторов.

B. Взять любое подходящее значение $\lambda$ и методом простых итераций найти приближенное решение этого уравнения
с указанной точностью $\varepsilon$, используя априорную оценку числа итераций.

C. Найти точное решение этого уравнения и сравнить с приближенным. </strong>

$x(t) = \lambda \int_0^1{\mathrm{(t^2 + s^2)x(s)}\,\mathrm{d}s} - 2t, \varepsilon = 10^{-4}$

<i>A. Найдем значения параметра $\lambda$, при которых к уравнению применим принцип сжимающих операторов.</i>

Рассмотрим интегральный оператор 
$$Ф: C[0; 1] \to C[0; 1], Ф[x] = \lambda \int_0^1{\mathrm{(t^2 + s^2)x(s)}\,\mathrm{d}s} - 2t.$$

Функция $K(t, s) = \lambda (t^2 + s^2)$ – ядро интегрального оператора. Уравнение $x(t) = \lambda \int_0^1{\mathrm{(t^2 + s^2)x(s)}\,\mathrm{d}s} - 2t$ имеет вид $Ф[x] = x$, его решение – неподвижная точка оператора $Ф$.
Обратимся к достаточному признаку сжимающего оператора. При условии
$$\alpha = \lambda \max_{t \in [0; 1]}{\int_0^1{\mathrm{|(t^2 + s^2)|}\,\mathrm{d}s}} < 1$$

оператор $Ф$ сжимающий с коэффициентом сжатия $\alpha$.

Необходимо вычислить $\int_0^1{\mathrm{|(t^2 + s^2)|}\,\mathrm{d}s} = \int_0^1{\mathrm{(t^2 + s^2)}\,\mathrm{d}s}$.

In [1]:
import sympy as sp
from IPython.display import *

sp.init_printing(use_latex='mathjax')

t = sp.Symbol('t')
s = sp.Symbol('s')

expr = t**2 + s**2
sp.integrate(expr, (s, 0, 1))


 2   1
t  + ─
     3

Находим максимум 
$$\max_{t \in [0; 1]}{(t^2 + \frac{1}{3})} = \frac{4}{3}$$

Осталось выразить параметр $\lambda$:
$$\alpha = \lambda \frac{4}{3} < 1$$

Таким образом, при $\lambda < \frac{3}{4}$ к уравнению 
$x(t) = \lambda \int_0^1{\mathrm{(t^2 + s^2)x(s)}\,\mathrm{d}s} - 2t$
применим принцип сжимающих операторов: уравнение имеет единственное решение и можно использовать метод простых итераций для поиска приближенного решения.

<i>B. Возьмем $\lambda = \frac{1}{2}$ и методом простых итераций найдем приближенное решение уравнения с точностью
$\varepsilon = 10^{-4}$ используя априорную оценку числа итераций.</i>

Для произвольного начального приближения $x_0$ последовательность итераций задается рекуррентной формулой $x_n = Ф[x_{n - 1}]$. В данном случае

$x_n(t) = \frac{1}{2} \int_0^1{\mathrm{(t^2 + s^2)x_{n - 1}(s)}\,\mathrm{d}s} - 2t$

Выбираем произвольным образом начальное приближение, например $x_0(t) = 1$, и вычисляем первую итерацию $x_1(t)$:

In [2]:
x_prev = sp.Symbol('x_{n - 1}')
under_integral_expr = (t**2 + s**2)*x_prev

x0 = 1
to_be_x1 = under_integral_expr.subs(x_prev, x0)

x1 = (sp.Rational(1, 2) * sp.integrate(to_be_x1, (s, 0, 1)) - 2*t)

display(x1)

 2          
t          1
── - 2⋅t + ─
2          6

Теперь найдем априорную оценку $N_{apr}$ числа итераций:

In [3]:
import math
from scipy.optimize import minimize_scalar
import numpy as np

def dist(x1, x0):
    rho_func = sp.lambdify(t, sp.Abs(x1 - x0))
    res = minimize_scalar(lambda x: -rho_func(x), bounds=[0,1], method='Bounded')
    return sp.Abs(x1 - x0).subs(t, res.x)

def is_enough(alp, x1, x0, eps): 
    return (alp/(1 - alp))*dist(x1, x0) <= eps

def apriori_weight(alp, eps, x0, x1):
    return (math.log(eps*(1 - alp)/dist(x0, x1), alp) + 1)

In [4]:
alpha = sp.Rational(1, 2)
varepsilon = 10**(-4)

apriori = apriori_weight(alpha, varepsilon, x0, x1)
display(apriori) 

16.510101115282527

Следовательно, для вычисления приближенного решения уравнения с точностью $\varepsilon = 10^{-4}$, достаточно провести 17 итераций:

In [5]:
x_next = x0
under_integral_expr = (t**2 + s**2)*x_prev

for i in range(17):
    if (i != 0):
        x_old = x_next.subs(t, s)
    else:
        x_old = x_next
    to_be_x1 = under_integral_expr.subs(x_prev, x_old)
    x_next = sp.Rational(1, 2) * sp.integrate(to_be_x1, (s, 0, 1)) - 2*t

display(x_next)

                  2                       
  56533585805663⋅t          20221629335392
- ───────────────── - 2⋅t - ──────────────
    67260501562500          50445376171875

<i>C. Найдем точное решение и сравним его с приближенным.</i>

Для интегрального уравнения Фредгольма с вырожденным ядром есть возможность найти точное решение.

$$x(t) = - 2t + \frac{1}{2} t^2 \int_0^1{\mathrm{x(s)}\,\mathrm{d}s} + \frac{1}{2} \int_0^1{\mathrm{s^2 x(s)}\,\mathrm{d}s}$$

Отсюда ясно, что решение имеет вид 
$$x(t) = -2t +c_1t^2 + c_2, c_1, c_2 \in R$$
$$c_1 = \frac{1}{2} \int_0^1{\mathrm{x(s)}\,\mathrm{d}s}$$
$$c_2 = \frac{1}{2} \int_0^1{\mathrm{s^2 x(s)}\,\mathrm{d}s}$$

Найдем значения коэффициентов $c_1$ и $c_2$:
$$c_1 = \frac{1}{2} \int_0^1{\mathrm{-2s +c_1s^2 + c_2}\,\mathrm{d}s}$$
$$c_2 = \frac{1}{2} \int_0^1{\mathrm{s^2 (-2s +c_1s^2 + c_2)}\,\mathrm{d}s}$$

In [6]:
c1 = sp.Symbol('c1')
c2 = sp.Symbol('c2')

expr1 = sp.Rational(1, 2)*(-2*s + c1*s**2 + c2)
expr2 = sp.Rational(1, 2)*(s**2)*(-2*s + c1*s**2 + c2)

display(sp.integrate(expr1, (s, 0, 1)) - c1)
display(sp.integrate(expr2, (s, 0, 1)) - c2)

  5⋅c₁   c₂   1
- ──── + ── - ─
   6     2    2

c₁   5⋅c₂   1
── - ──── - ─
10    6     4

После вычисления интегралов необходимо решить систему двух линейных алгебраических уравнений с двумя неизвестными
$$\begin{cases} 
\\- \frac{5 c_{1}}{6} + \frac{c_{2}}{2} = \frac{1}{2}
\\ \frac{c_{1}}{10} - \frac{5 c_{2}}{6} = \frac{1}{4}
\end{cases}$$

In [7]:
import numpy as np

a = np.array([[-5/6, 0.5],
              [0.1, -5/6]])

b = np.array([[0.5],
              [0.25]])

x_exact = np.linalg.solve(a, b)
display(x_exact)

array([[-0.84051724],
       [-0.40086207]])

Сравним точное решение уравнения с приближенным решением $x_{17}(t)$ в метрике пространства $C[0; 1]$:

In [8]:
exact = -2*t + x_exact[0][0]*t**2 + x_exact[1][0]
dist(exact, x_next) < varepsilon

True

Таким образом, приближенное решение обладает требуемой точностью.